# Model Exercise
<hr style="border:2px solid red"> </hr>
<hr style="border:2px solid red"> </hr>

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import acquire

from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns


### Using the titanic data, do the following:
### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.



In [2]:
# Acquiring my titanic dataset
titanic_train, titanic_validate, titanic_test = acquire.prep_titanic(acquire.get_titanic_data()) 
titanic_train.head()

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embark_town,alone,is_female,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
572,572,1,1,male,0,0,26.3875,Southampton,1,0,0,0,1
430,430,1,1,male,0,0,26.5500,Southampton,1,0,0,0,1
858,858,1,3,female,0,3,19.2583,Cherbourg,0,1,1,0,0
186,186,1,3,female,1,0,15.5000,Queenstown,0,1,0,1,0
206,206,0,3,male,1,0,15.8500,Southampton,0,0,0,0,1


In [3]:
# YOU CAN'T HAVE ANY STRING COLUMNS!!!
# Drop the original columns we encoded
titanic_train = titanic_train.drop(columns=["sex", "embark_town"])
titanic_validate = titanic_validate.drop(columns=["sex", "embark_town"])
titanic_test = titanic_test.drop(columns=["sex", "embark_town"])


In [4]:
# Okay, now to figure out baseline predictions
titanic_train.survived.value_counts()

0    307
1    191
Name: survived, dtype: int64

In [5]:
# I clearly want 'did not survive' or '0' to be my baseline prediction
# since there are only two values, the highest value count will also be the mode!

baseline = (titanic_train.survived.mode() )[0] # I designated element because of what it returns without designation

baseline_accuracy = ( titanic_train.survived == baseline ).mean()
print("Baseline accuracy: ", round(baseline_accuracy * 100, 2), "%")

Baseline accuracy:  61.65 %


> <b>Answer 1:</b>
>
> <b>Baseline Prediction: '0' which means they did not survive</b>
>
> <b>Baseline Accuracy: 61.65%<b/>


### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)




In [6]:
# First I need to create the X & y version of train
# where y is a series with just the target variable and X are all the features. 

X_train = titanic_train.drop(columns=["survived"])
y_train = titanic_train.survived

X_validate = titanic_validate.drop(columns=["survived"])
y_validate = titanic_validate.survived

X_test = titanic_test.drop(columns=["survived"])
y_test = titanic_test.survived


In [7]:
# Visualize 
X_train.head()

,passenger_id,pclass,sibsp,parch,fare,alone,is_female,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
572,572,1,0,0,26.3875,1,0,0,0,1
430,430,1,0,0,26.5500,1,0,0,0,1
858,858,3,0,3,19.2583,0,1,1,0,0
186,186,3,1,0,15.5000,0,1,0,1,0
206,206,3,1,0,15.8500,0,0,0,0,1


In [8]:
# Visualize 
y_train.head()

572    1
430    1
858    1
186    1
206    0
Name: survived, dtype: int64

In [9]:
# Training the Model

# Create the Decision Tree object with desired hyper-parameters.
# for classification you can change the algorithm to gini or entropy (information gain).  
# Default is gini.

classifier = DecisionTreeClassifier(max_depth=3, random_state=123)

In [10]:
# Fit the model 
classifier = classifier.fit(X_train, y_train)

In [11]:
# Make Predictions/ Use the model
# This is where the model makes predictions, based on training set obeservations
y_predictions = classifier.predict(X_train)
y_predictions[0:5]

array([1, 0, 1, 1, 0])

In [12]:
# Estimate Probability
# Estimate the probability of each passenger's survival, using the training data.
predict_probability = classifier.predict_proba(X_train)
predict_probability[0:5]

array([[0.4       , 0.6       ],
       [0.8       , 0.2       ],
       [0.38235294, 0.61764706],
       [0.38235294, 0.61764706],
       [0.89855072, 0.10144928]])

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.



In [13]:
# Evaluate Model
# Compute the Accuracy

# Remember that Accuracy is the number of correct predictions over the number of total instances 
# that have been evaluated.

print('Accuracy of Decision Tree classifier on training set: {:.2f}%'
      .format(classifier.score(X_train, y_train) * 100 ))

Accuracy of Decision Tree classifier on training set: 81.73%


In [14]:
# Confusion Matrix 
# The rows are whether the passenger actual survived (in the y_train)
# The columns are what the y_predictions predicted 

pd.DataFrame(confusion_matrix(y_train, y_predictions))

,0,1
0,258,49
1,42,149


In [15]:
confusion_matrix(y_train, y_predictions)[0][1]

49

In [16]:
# Classification Report 
report = classification_report(y_train, y_predictions, output_dict=True)
pd.DataFrame(report).T

,precision,recall,f1-score,support
0,0.860000,0.840391,0.850082,307.000000
1,0.752525,0.780105,0.766067,191.000000
accuracy,0.817269,0.817269,0.817269,0.817269
macro avg,0.806263,0.810248,0.808075,498.000000
weighted avg,0.818780,0.817269,0.817860,498.000000


### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.



In [17]:
# Positive: 'did not survive'
# Negative: 'survived'

# Assign values from Confusion Matrix:
TP = 258
FP = 49
FN = 42
TN = 149

# Computations:
accuracy = (TP + TN)/(TP + FP + FN + TN)
print("Accuracy:", round(accuracy*100,2))

true_positive = TP/(TP+FN)
print("True Positive:", round(true_positive*100, 2) , "%")

false_positive = FP/(FP+TN)
print("False Positive:", round(false_positive*100, 2) , "%")

true_negative = TN/(TN+FP)
print("True Negative:", round(true_negative*100, 2) , "%")

false_negative = FN/(FN+TP)
print("False Negative:", round(false_negative*100, 2) , "%")

precision = TP/(TP+FP)
print("Precision:", round(precision*100, 2) , "%")

recall = TP/(TP+FN)
print("Recall:", round(recall*100, 2) , "%")

# f1-score: The balanced harmonic mean of Recall and Precision, giving both metrics equal weight. 
# The higher the F-Measure is, the better. 
f1_score = 2*(precision*recall)/(precision+recall)
print("F1-Score:", round(f1_score*100, 2) , "%")

# Support: number of occurrences of each class in where y is true.
pos_support = TP + FN
print("Positive Support:", pos_support)

neg_support = FP + TN
print("Negative Support:", neg_support)


Accuracy: 81.73
True Positive: 86.0 %
False Positive: 24.75 %
True Negative: 75.25 %
False Negative: 14.0 %
Precision: 84.04 %
Recall: 86.0 %
F1-Score: 85.01 %
Positive Support: 300
Negative Support: 198


### 5. Run through steps 2-4 using a different max_depth value.



In [18]:
# Yay! It's looping season!

for i in range(1, 16):
    # Train the model
    classifier = DecisionTreeClassifier(max_depth= i , random_state=123)
    
    # Fit the model 
    classifier = classifier.fit(X_train, y_train)

    # Make Predictions
    # This is where the model makes predictions, based on training set obeservations
    y_predictions = classifier.predict(X_train)
    y_predictions[0:5]

    # Produce the classification report on the actual y values and this model's predicted y values
    report = classification_report(y_train, y_predictions, output_dict=True)
    print("Tree with max depth of", i )
    print(pd.DataFrame(report))
    print()
    
    # Evaluate the Model
    print('Accuracy of Decision Tree classifier on training set: {:.2f}%'
          .format(classifier.score(X_train, y_train) * 100 ))
    
    # Confusion Matrix 
    pd.DataFrame(confusion_matrix(y_train, y_predictions))
    
    # Classification Report 
    report = classification_report(y_train, y_predictions, output_dict=True)
    pd.DataFrame(report).T
    
    # Assign values from Confusion Matrix:
    TP = confusion_matrix(y_train, y_predictions)[0][0]
    FP = confusion_matrix(y_train, y_predictions)[0][1]
    FN = confusion_matrix(y_train, y_predictions)[1][0]
    TN = confusion_matrix(y_train, y_predictions)[1][1]

    # Computations:
    accuracy = (TP + TN)/(TP + FP + FN + TN)
    print("Accuracy:", round(accuracy*100,2))

    true_positive = TP/(TP+FN)
    print("True Positive:", round(true_positive*100, 2) , "%")

    false_positive = FP/(FP+TN)
    print("False Positive:", round(false_positive*100, 2) , "%")

    true_negative = TN/(TN+FP)
    print("True Negative:", round(true_negative*100, 2) , "%")

    false_negative = FN/(FN+TP)
    print("False Negative:", round(false_negative*100, 2) , "%")

    precision = TP/(TP+FP)
    print("Precision:", round(precision*100, 2) , "%")

    recall = TP/(TP+FN)
    print("Recall:", round(recall*100, 2) , "%")

    # f1-score: The balanced harmonic mean of Recall and Precision, giving both metrics equal weight. 
    # The higher the F-Measure is, the better. 
    f1_score = 2*(precision*recall)/(precision+recall)
    print("F1-Score:", round(f1_score*100, 2) , "%")

    # Support: number of occurrences of each class in where y is true.
    pos_support = TP + FN
    print("Positive Support:", pos_support)

    neg_support = FP + TN
    print("Negative Support:", neg_support)

    print("===============================================================================")

Tree with max depth of 1
                    0           1  accuracy   macro avg  weighted avg
precision    0.800000    0.728324    0.7751    0.764162      0.772510
recall       0.846906    0.659686    0.7751    0.753296      0.775100
f1-score     0.822785    0.692308    0.7751    0.757546      0.772742
support    307.000000  191.000000    0.7751  498.000000    498.000000

Accuracy of Decision Tree classifier on training set: 77.51%
Accuracy: 77.51
True Positive: 80.0 %
False Positive: 27.17 %
True Negative: 72.83 %
False Negative: 20.0 %
Precision: 84.69 %
Recall: 80.0 %
F1-Score: 82.28 %
Positive Support: 325
Negative Support: 173
Tree with max depth of 2
                    0           1  accuracy   macro avg  weighted avg
precision    0.800000    0.728324    0.7751    0.764162      0.772510
recall       0.846906    0.659686    0.7751    0.753296      0.775100
f1-score     0.822785    0.692308    0.7751    0.757546      0.772742
support    307.000000  191.000000    0.7751  498.00000

Accuracy: 99.0
True Positive: 99.67 %
False Positive: 2.06 %
True Negative: 97.94 %
False Negative: 0.33 %
Precision: 98.7 %
Recall: 99.67 %
F1-Score: 99.18 %
Positive Support: 304
Negative Support: 194
Tree with max depth of 13
                    0           1  accuracy   macro avg  weighted avg
precision    0.993506    0.994737  0.993976    0.994122      0.993978
recall       0.996743    0.989529  0.993976    0.993136      0.993976
f1-score     0.995122    0.992126  0.993976    0.993624      0.993973
support    307.000000  191.000000  0.993976  498.000000    498.000000

Accuracy of Decision Tree classifier on training set: 99.40%
Accuracy: 99.4
True Positive: 99.35 %
False Positive: 0.53 %
True Negative: 99.47 %
False Negative: 0.65 %
Precision: 99.67 %
Recall: 99.35 %
F1-Score: 99.51 %
Positive Support: 308
Negative Support: 190
Tree with max depth of 14
                    0           1  accuracy   macro avg  weighted avg
precision    0.996753    1.000000  0.997992    0.998377    

### 6. Which model performs better on your in-sample data?



> <b>Any max depth >= 15 has a accuracy of 100%</b>
>
> <b>So if I wanted to avoid overfitting I might go more with a max depth of 11, where the accuracy is still 98.19%</b>


### 7. Which model performs best on your out-of-sample data, the validate set?

In [19]:
# Yay! It's looping season!

for i in range(1, 16):
    # Train the model
    classifier = DecisionTreeClassifier(max_depth= i , random_state=123)
    
    # Fit the model 
    classifier = classifier.fit(X_validate, y_validate)

    # Make Predictions
    # This is where the model makes predictions, based on training set obeservations
    y_predictions = classifier.predict(X_validate)
    y_predictions[0:5]

    # Produce the classification report on the actual y values and this model's predicted y values
    report = classification_report(y_validate, y_predictions, output_dict=True)
    print("Tree with max depth of", i )
    print(pd.DataFrame(report))
    print()
    
    # Evaluate the Model
    print('Accuracy of Decision Tree classifier on training set: {:.2f}%'
          .format(classifier.score(X_validate, y_validate) * 100 ))
    
    # Confusion Matrix 
    pd.DataFrame(confusion_matrix(y_validate, y_predictions))
    
    # Classification Report 
    report = classification_report(y_validate, y_predictions, output_dict=True)
    pd.DataFrame(report).T
    
    # Assign values from Confusion Matrix:
    TP = confusion_matrix(y_validate, y_predictions)[0][0]
    FP = confusion_matrix(y_validate, y_predictions)[0][1]
    FN = confusion_matrix(y_validate, y_predictions)[1][0]
    TN = confusion_matrix(y_validate, y_predictions)[1][1]

    # Computations:
    accuracy = (TP + TN)/(TP + FP + FN + TN)
    print("Accuracy:", round(accuracy*100,2))

    true_positive = TP/(TP+FN)
    print("True Positive:", round(true_positive*100, 2) , "%")

    false_positive = FP/(FP+TN)
    print("False Positive:", round(false_positive*100, 2) , "%")

    true_negative = TN/(TN+FP)
    print("True Negative:", round(true_negative*100, 2) , "%")

    false_negative = FN/(FN+TP)
    print("False Negative:", round(false_negative*100, 2) , "%")

    precision = TP/(TP+FP)
    print("Precision:", round(precision*100, 2) , "%")

    recall = TP/(TP+FN)
    print("Recall:", round(recall*100, 2) , "%")

    # f1-score: The balanced harmonic mean of Recall and Precision, giving both metrics equal weight. 
    # The higher the F-Measure is, the better. 
    f1_score = 2*(precision*recall)/(precision+recall)
    print("F1-Score:", round(f1_score*100, 2) , "%")

    # Support: number of occurrences of each class in where y is true.
    pos_support = TP + FN
    print("Positive Support:", pos_support)

    neg_support = FP + TN
    print("Negative Support:", neg_support)

    print("===============================================================================")

Tree with max depth of 1
                    0          1  accuracy   macro avg  weighted avg
precision    0.823529   0.743590  0.794393    0.783560      0.792898
recall       0.848485   0.707317  0.794393    0.777901      0.794393
f1-score     0.835821   0.725000  0.794393    0.780410      0.793357
support    132.000000  82.000000  0.794393  214.000000    214.000000

Accuracy of Decision Tree classifier on training set: 79.44%
Accuracy: 79.44
True Positive: 82.35 %
False Positive: 25.64 %
True Negative: 74.36 %
False Negative: 17.65 %
Precision: 84.85 %
Recall: 82.35 %
F1-Score: 83.58 %
Positive Support: 136
Negative Support: 78
Tree with max depth of 2
                    0          1  accuracy   macro avg  weighted avg
precision    0.764706   0.954545  0.803738    0.859626      0.837448
recall       0.984848   0.512195  0.803738    0.748522      0.803738
f1-score     0.860927   0.666667  0.803738    0.763797      0.786491
support    132.000000  82.000000  0.803738  214.000000    214

support    132.0  82.0       1.0      214.0         214.0

Accuracy of Decision Tree classifier on training set: 100.00%
Accuracy: 100.0
True Positive: 100.0 %
False Positive: 0.0 %
True Negative: 100.0 %
False Negative: 0.0 %
Precision: 100.0 %
Recall: 100.0 %
F1-Score: 100.0 %
Positive Support: 132
Negative Support: 82



> <b>Any max depth >= 12 has a accuracy of 100%</b>
>
> <b>So if I wanted to avoid overfitting I might go more with a max depth of 1, where the accuracy is still 97.20%</b>